<a href="https://colab.research.google.com/github/M-Nitsche/BeeWatch/blob/main/dataset/flickr_dataset_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collect dataset by flickr website

In [1]:
!pip install flickrapi

     |████████████████████████████████| 61kB 6.0MB/s 


In [2]:
from flickrapi import FlickrAPI

In [3]:
KEY = 'KEY'
SECRET = 'SECRET'

SIZES = ["url_o", "url_k", "url_h", "url_l", "url_c"]  # in order of preference